In [2]:
import time
import ast
import traceback
import random
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd

import selenium
import webdriver_manager
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException

from crawler import get_url, click_elm
from test import QuantusQA
from settings import filter_conditions

from database import AccessDataBase
db = AccessDataBase("quantus_test")

from settings import filter_conditions
from test import QuantusQA
qqa = QuantusQA()


---
#### V1

In [3]:
import os, sys, time, shutil

variables_v1 = {}
def read_result(kind):
    for p in sys.path:
        if "Users" in p:
            user_name = p.split("/")[2]    
            download_dir = f"/Users/{user_name}/Downloads"
            bak_dir = os.path.join(download_dir, "bak")
            if not os.path.exists(bak_dir):
                os.mkdir(bak_dir)
            break

    files = os.listdir(download_dir)
    files = [os.path.join(download_dir, f) for f in files]
    files.sort(key=lambda x: os.path.getmtime(x))
    
    strategy_name = variables_v1["strategy_name"]
    
    if kind == 0:
        # backtest
        # file = strategy_name + "_backtest_result.html"
        file = strategy_name + ".html"
        path = os.path.join(download_dir, file)
        bak = os.path.join(bak_dir, file)
        
        # 결과 파일 생성 여부 확인
        ck = True
        st = time.time()
        while not os.path.exists(path):
            time.sleep(10)
            # 백테스트에 100초 이상 소요되면 에러로 간주
            if time.time() - st > 100:
                ck = False
                print("backtest error")
                break

        if ck:
            
            _wd = get_url("file://" + path)
            soup = BeautifulSoup(_wd.page_source, "lxml")
            tbl = soup.find("div", {"id": "right"}).find("table")
            tbl_trs = tbl.find_all("tr")

            # Cumulative Return
            tbl_tr_td = tbl_trs[4].find_all("td")
            variables_v1[tbl_tr_td[0].text] = tbl_tr_td[1].text
            # CAGR
            tbl_tr_td = tbl_trs[5].find_all("td")
            variables_v1[tbl_tr_td[0].text] = tbl_tr_td[1].text
            # Max Drawdown	
            tbl_tr_td = tbl_trs[15].find_all("td")
            variables_v1[tbl_tr_td[0].text] = tbl_tr_td[1].text
            
            shutil.move(path, bak)
            _wd.quit()
            
            
            
    elif kind == 1:
        # port extraction
        # file = strategy_name + "_portfolio_result.csv"
        file = "port_result.csv"
        path = os.path.join(download_dir, file)
        bak = os.path.join(bak_dir, file)
        
        # 결과 파일 생성 여부 확인
        ck = True
        st = time.time()            
        while not os.path.exists(path):
            time.sleep(10)
            # 백테스트에 100초 이상 소요되면 에러로 간주
            if time.time() - st > 60:
                print("port extraction error")
                ck = False
                break
        
        if ck:
            df = pd.read_csv(path)
            variables_v1["portfolio"] = str(df["Name"].tolist())
            variables_v1["industry"] = str(df["업종명"].tolist())
            shutil.move(path, bak)
            
            
    elif kind == 2:
        # 10분위 테스트
        # file = strategy_name + "_quantile_result.csv"
        file = "quantile_result.csv"
        path = os.path.join(download_dir, file)
        bak = os.path.join(bak_dir, file)
        
        # 결과 파일 생성 여부 확인
        ck = True
        st = time.time()            
        while not os.path.exists(path):
            time.sleep(10)
            # 10분위 테스트에 180초 이상 소요되면 에러로 간주
            if time.time() - st > 180:
                print("decile test error")
                ck = False
                break
        
        if ck:
            df = pd.read_csv(path)
            
            data = df.iloc[4, :].tolist()
            variables_v1[data[0].strip() + "s"] = data[1:]

            data = df.iloc[5, :].tolist()
            variables_v1[data[0].strip() + "s"] = data[1:]

            data = df.iloc[10, :].tolist()
            variables_v1[data[0].strip() + "s"] = data[1:]
            shutil.move(path, bak)
            
    else:
        raise TypeError("올바른 kind(0: backtest, 1: port, 2: decile)를 입력하세요")

In [12]:
variables_v1["strategy_name"] = "신마법공식"

read_result(kind=0)
read_result(kind=1)
read_result(kind=2)
variables_v1["version"] = "V1"

{'strategy_name': '신마법공식',
 'Cumulative Return': '13,713.54%',
 'CAGR': '27.46%',
 'Max Drawdown': '-62.46%',
 'portfolio': "['KTis', 'KTcs', '스카이라이프', '대현', 'LF', '슈피겐코리아', 'NHN', '신영와코루', '현대홈쇼핑', '한섬', 'KT', 'LG헬로비전', 'LG유플러스', '현대백화점', '태평양물산', 'HMM', '호전실업', '이마트', 'AJ네트웍스', '현대리바트']",
 'industry': "['상사,자본재', '상사,자본재', '미디어,교육', '화장품,의류,완구', '화장품,의류,완구', 'IT하드웨어', '소프트웨어', '화장품,의류,완구', '소매(유통)', '화장품,의류,완구', '통신서비스', '미디어,교육', '통신서비스', '소매(유통)', '화장품,의류,완구', '운송', '화장품,의류,완구', '소매(유통)', '운송', '건설,건축관련']",
 'Cumulative Returns': ['7,774.60',
  '6,243.70',
  '4,028.50',
  '1,598.35',
  '1,298.87',
  '720.61',
  '663.44',
  '242.59',
  '100.65',
  '-54.7'],
 'CAGRs': ['23.98',
  '22.67',
  '20.1',
  '14.96',
  '13.87',
  '10.92',
  '10.53',
  '6.25',
  '3.49',
  '-3.82'],
 'Max Drawdowns': ['-54.33',
  '-61.11',
  '-57.73',
  '-59.25',
  '-58.67',
  '-61.37',
  '-59.36',
  '-65.01',
  '-68.19',
  '-79.05'],
 'version': 'V1'}

In [ ]:
# variables_list = [variables_v1]
# result_df_v1 = pd.DataFrame(variables_list)
# result_df_v1 = result_df_v1.astype("str")
# result_df_v1

---
#### V2

In [4]:
wd = qqa.init_page(**filter_conditions)

In [4]:
qqa.variables["strategy_name"] = "신마법공식"
qqa.read_result(wd, kind=0, start_time=time.time())
qqa.read_result(wd, kind=1, start_time=time.time())
qqa.read_result(wd, kind=2, start_time=time.time())
qqa.variables["version"] = "V2"

In [19]:
for key in variables_v1.keys():
    v1 = variables_v1[key]
    v2 = qqa.variables[key]
    
    if v1 != v2:
        print(f"{key}\n- v1: {v1}\n- v2: {v2}\n\n")

Cumulative Return
- v1: 13,713.54%
- v2: 15,924.22%


CAGR
- v1: 27.46%
- v2: 29.41%


portfolio
- v1: ['KTis', 'KTcs', '스카이라이프', '대현', 'LF', '슈피겐코리아', 'NHN', '신영와코루', '현대홈쇼핑', '한섬', 'KT', 'LG헬로비전', 'LG유플러스', '현대백화점', '태평양물산', 'HMM', '호전실업', '이마트', 'AJ네트웍스', '현대리바트']
- v2: ['KTis', 'KTcs', '스카이라이프', '대현', 'LF', 'NHN', '슈피겐코리아', '신영와코루', '현대홈쇼핑', 'KT', '한섬', 'LG유플러스', 'LG헬로비전', '현대백화점', 'HMM', '태평양물산', '이마트', '호전실업', 'AJ네트웍스', '현대리바트']


industry
- v1: ['상사,자본재', '상사,자본재', '미디어,교육', '화장품,의류,완구', '화장품,의류,완구', 'IT하드웨어', '소프트웨어', '화장품,의류,완구', '소매(유통)', '화장품,의류,완구', '통신서비스', '미디어,교육', '통신서비스', '소매(유통)', '화장품,의류,완구', '운송', '화장품,의류,완구', '소매(유통)', '운송', '건설,건축관련']
- v2: ['상사,자본재', '상사,자본재', '미디어,교육', '화장품,의류,완구', '화장품,의류,완구', '소프트웨어', 'IT하드웨어', '화장품,의류,완구', '소매(유통)', '통신서비스', '화장품,의류,완구', '통신서비스', '미디어,교육', '소매(유통)', '운송', '화장품,의류,완구', '소매(유통)', '화장품,의류,완구', '운송', '건설,건축관련']


Cumulative Returns
- v1: ['7,774.60', '6,243.70', '4,028.50', '1,598.35', '1,298.87', '720.61', '663.44', '242.59', '1

In [13]:
qqa.variables

{'custom_filters': [],
 'custom_factors': [],
 'logs': [],
 'alert_msg': [],
 'errors': [],
 'level': '중급',
 'login': '카카오',
 'strategy_name': '신마법공식',
 'backtest_elapsed': 0.0,
 'Cumulative Return': '15,924.22%',
 'CAGR': '29.41%',
 'Max Drawdown': '-62.46%',
 'port_elapsed': 0.0,
 'portfolio': "['KTis', 'KTcs', '스카이라이프', '대현', 'LF', 'NHN', '슈피겐코리아', '신영와코루', '현대홈쇼핑', 'KT', '한섬', 'LG유플러스', 'LG헬로비전', '현대백화점', 'HMM', '태평양물산', '이마트', '호전실업', 'AJ네트웍스', '현대리바트']",
 'industry': "['상사,자본재', '상사,자본재', '미디어,교육', '화장품,의류,완구', '화장품,의류,완구', '소프트웨어', 'IT하드웨어', '화장품,의류,완구', '소매(유통)', '통신서비스', '화장품,의류,완구', '통신서비스', '미디어,교육', '소매(유통)', '운송', '화장품,의류,완구', '소매(유통)', '화장품,의류,완구', '운송', '건설,건축관련']",
 'decile_elapsed': 0.0,
 'Cumulative Returns': ['9,616.74',
  '7,299.24',
  '5,297.81',
  '2,151.02',
  '1,783.49',
  '1,071.54',
  '1,008.87',
  '485.35',
  '247.96',
  '-16.22'],
 'CAGRs': ['26.16',
  '24.43',
  '22.45',
  '17.13',
  '16.08',
  '13.31',
  '12.99',
  '9.39',
  '6.54',
  '-0.89'],
 'Max Drawd

In [ ]:
wd.quit()